In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

# Load the TFLite model
model_path = "manipulation_detector_config_4.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Preprocesses an image before it is given to the model to ensure consistency
# across all images that the model is learning off of.
def preprocess_image(image_path):
    # Load the image and convert it to RGB colour space; 
    # we want all images to be in same colour space for consistency.
    with Image.open(image_path) as image:
        image = image.convert("RGB")
    image = image.resize((384, 384))
    # Convert the image to an array. Normalize its pixel values to be between 0 and 1.
    # Since most images in the training/test dataset are jpegs and in RGB, assume that
    # we can divide each pixel by 255 to normalize them to be between 0 and 1.
    preprocessed_image = np.array(image) / 255.0
    # Return the preprocessed image.
    return preprocessed_image

# Define a function to classify a single image
def classify_image(image_path):
    input_data = preprocess_image(image_path)
    input_data = np.expand_dims(input_data, axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data[0][0]

# Define the folder where the images are located
folder_path = "reserve"

# Initialize counters for classification results
num_authentic = 0
num_manipulated = 0
num_au_authentic = 0
num_au_manipulated = 0
num_tp_authentic = 0
num_tp_manipulated = 0
num_au_filenames = 0
num_tp_filenames = 0

# Iterate over all the images in the folder
for filename in os.listdir(folder_path):
    if (filename.endswith(".jpg") or filename.endswith(".jpeg")):
        # Classify the image and update counters
        image_path = os.path.join(folder_path, filename)
        classification_result = classify_image(image_path)
        if classification_result < 0.5:
            print(f"{filename} - Manipulated: {classification_result}")
            num_manipulated += 1
            if (filename.startswith("Au") or filename.startswith("au")):
                num_au_manipulated += 1
                num_au_filenames += 1
            elif (filename.startswith("Tp") or filename.startswith("tp")):
                num_tp_manipulated += 1
                num_tp_filenames += 1
        else:
            print(f"{filename} - Authentic: {classification_result}")
            num_authentic += 1
            if (filename.startswith("Au") or filename.startswith("au")):
                num_au_authentic += 1
                num_au_filenames += 1
            elif (filename.startswith("Tp") or filename.startswith("tp")):
                num_tp_authentic += 1
                num_tp_filenames += 1

# Print the classification results and filename counts
print(f"Total authentic images: {num_authentic}")
print(f"Total manipulated images: {num_manipulated}")
print(f"Number of 'au' authentic images: {num_au_authentic}")
print(f"Number of 'au' manipulated images: {num_au_manipulated}")
print(f"Number of 'tp' authentic images: {num_tp_authentic}")
print(f"Number of 'tp' manipulated images: {num_tp_manipulated}")
print(f"Number of 'au' filenames: {num_au_filenames}")
print(f"Number of 'tp' filenames: {num_tp_filenames}")

precision = num_au_authentic/(num_au_authentic + num_tp_authentic)
recall = num_au_authentic/(num_au_authentic + num_au_manipulated)
accuracy = (num_au_authentic + num_tp_manipulated)/(num_au_authentic + num_tp_authentic + num_au_manipulated + num_tp_manipulated)

print(f"precision: {precision}")
print(f"recall: {recall}")
print(f"accuracy: {accuracy}")



Au_pla_30535.jpg - Authentic: 1.0
Tp_S_NRD_S_N_pla20018_pla20018_02489.jpeg - Manipulated: 0.000560843909624964
Au_pla_30253.jpg - Manipulated: 0.016410572454333305
Au_pla_20040.jpg - Manipulated: 0.0010920619824901223
Au_pla_30247.jpg - Manipulated: 0.034795477986335754
Au_pla_20054.jpg - Manipulated: 5.92820651945658e-06
Au_pla_30521.jpg - Authentic: 0.9999471306800842
Tp_S_NRN_S_N_pla00065_pla00065_01172.jpeg - Authentic: 0.9894380569458008
Au_pla_30509.jpg - Authentic: 1.0
Au_arc_30592.jpg - Authentic: 0.9999997615814209
Tp_S_NRN_S_N_ind00055_ind00055_00905.jpeg - Authentic: 1.0
Tp_D_CRN_S_N_sec00071_art00028_11281.jpg - Manipulated: 4.354472548584454e-05
Tp_D_NNN_L_N_ani00010_nat00095_10218.jpeg - Authentic: 0.9988635182380676
Au_pla_20068.jpg - Manipulated: 0.02354556694626808
Au_arc_30586.jpg - Authentic: 1.0
Au_nat_30570.jpg - Authentic: 0.9985414147377014
Au_nat_30216.jpg - Authentic: 0.9999582171440125
Au_nat_20005.jpg - Manipulated: 2.2002743207849562e-05
Au_pla_30290.jpg - 

Au_pla_30483.jpg - Authentic: 0.901269793510437
Tp_D_CND_M_N_ani00018_sec00096_00138.jpeg - Authentic: 0.9999698996543884
Tp_S_NRN_S_N_sec00014_sec00014_11232.jpg - Manipulated: 0.006436901167035103
Au_nat_30363.jpg - Authentic: 0.999993085861206
Au_nat_30388.jpg - Authentic: 0.9999939799308777
Au_sec_30181.jpg - Authentic: 0.9999948740005493
Au_sec_00042.jpg - Authentic: 0.8506563305854797
Au_pla_30468.jpg - Authentic: 0.857905924320221
Au_sec_30195.jpg - Authentic: 0.9999999403953552
Au_sec_00056.jpg - Authentic: 0.9997825622558594
Au_pla_30332.jpg - Manipulated: 0.04889410734176636
Tp_D_NNN_M_B_nat00004_nat00095_11488.jpg - Manipulated: 0.0027998429723083973
Tp_D_CRN_M_N_ani10112_ani00100_11648.jpg - Manipulated: 0.12569408118724823
Au_pla_30454.jpg - Manipulated: 0.43511155247688293
Au_cha_00062.jpg - Authentic: 0.9998716711997986
Au_pla_30440.jpg - Authentic: 0.9999982714653015
Au_pla_30326.jpg - Manipulated: 0.3221956789493561
Au_sec_30426.jpg - Authentic: 0.9964549541473389
Au_a

Au_cha_30028.jpg - Authentic: 0.999998152256012
Au_arc_30546.jpg - Authentic: 1.0
Au_sec_30034.jpg - Authentic: 1.0
Au_sec_30020.jpg - Authentic: 0.9986892342567444
Au_nat_30229.jpg - Authentic: 0.9999669194221497
Tp_D_CRN_S_N_ind00006_art00076_11745.jpg - Manipulated: 0.0009645733516663313
Au_arc_30552.jpg - Authentic: 0.9998931884765625
Tp_S_NRN_S_N_art00082_art00082_10285.jpeg - Authentic: 0.9999682903289795
Au_pla_30046.jpg - Manipulated: 0.1868060827255249
Tp_S_NRN_S_N_pla20083_pla20083_01983.jpeg - Manipulated: 0.00012699035869445652
Au_pla_30720.jpg - Authentic: 0.9998914003372192
Au_pla_30734.jpg - Authentic: 0.9999426007270813
Au_pla_30052.jpg - Authentic: 0.9928357005119324
Au_sec_30593.jpg - Authentic: 1.0
Au_arc_30787.jpg - Authentic: 1.0
Au_arc_30793.jpg - Authentic: 0.9999989867210388
Tp_D_CRN_S_N_nat00099_ani00017_10220.jpeg - Authentic: 0.980778157711029
Au_sec_30587.jpg - Authentic: 0.9999961853027344
Au_pla_30708.jpg - Authentic: 0.999921977519989
Tp_S_NRN_S_N_arc2008

Tp_D_CRN_S_N_arc00008_ani00070_11774.jpg - Manipulated: 0.0005171011434867978
Au_sec_00069.jpg - Authentic: 0.9939984083175659
Au_pla_30325.jpg - Manipulated: 0.0027511927764862776
Tp_S_NRN_S_N_ani00003_ani00003_00123.jpeg - Authentic: 0.9999234080314636
Tp_S_NRN_S_N_ind00036_ind00036_00450.jpeg - Authentic: 1.0
Au_pla_30443.jpg - Authentic: 0.9999809861183167
Au_cha_00075.jpg - Authentic: 0.9997461438179016
Au_nat_10140.jpeg - Manipulated: 0.0004970483132638037
Tp_D_CNN_S_N_nat00060_nat00060_00667.jpeg - Manipulated: 0.11233098804950714
Au_pla_30457.jpg - Authentic: 0.9910344481468201
Au_cha_00061.jpg - Authentic: 0.9954675436019897
Au_pla_30331.jpg - Manipulated: 0.0035752265248447657
Tp_S_NRN_S_N_ani20039_ani20039_02082.jpeg - Authentic: 0.9447776079177856
Au_arc_30625.jpg - Authentic: 0.9999989867210388
Tp_D_CRN_M_N_nat00084_nat00099_10071.jpeg - Authentic: 0.9969013333320618
Au_nat_30638.jpg - Authentic: 0.9999995827674866
Au_sec_30431.jpg - Authentic: 0.9999978542327881
Au_sec_30

Au_sec_30025.jpg - Authentic: 0.9999915361404419
Tp_S_NRN_S_N_art00019_art00019_10276.jpeg - Authentic: 0.9992898106575012
Au_pla_20091.jpg - Manipulated: 0.1780707836151123
Au_pla_30282.jpg - Manipulated: 0.027029620483517647
Au_nat_20017.jpg - Manipulated: 0.003620646195486188
Au_nat_30204.jpg - Authentic: 0.9999848008155823
Au_nat_30562.jpg - Authentic: 0.9950672388076782
Tp_S_NRN_S_N_pla20022_pla20022_01922.jpeg - Manipulated: 0.00011374626046745107
Au_nat_30576.jpg - Authentic: 0.9999999403953552
Au_nat_20003.jpg - Manipulated: 0.006697461940348148
Au_nat_30210.jpg - Authentic: 0.999993622303009
Au_pla_20085.jpg - Manipulated: 1.0949874194920994e-05
Au_pla_30296.jpg - Authentic: 0.9999966025352478
Au_sec_30019.jpg - Authentic: 0.9999999403953552
Au_sec_30596.jpg - Authentic: 0.999999463558197
Au_pla_30719.jpg - Authentic: 0.9999766945838928
Au_arc_30782.jpg - Authentic: 1.0
Au_arc_30796.jpg - Authentic: 0.9999966025352478
Au_sec_30582.jpg - Authentic: 1.0
Au_pla_30725.jpg - Authen

...

Tp_D_CRN_S_N_cha00063_art00014_11818.jpg - Manipulated: 2.1264009774313308e-05
Au_pla_30661.jpg - Authentic: 0.9999738335609436
Tp_D_CRN_S_N_art00067_art00092_11808.jpg - Manipulated: 0.01188186090439558
Tp_S_NRN_M_N_cha10208_cha10208_12338.jpg - Manipulated: 0.49296215176582336
Au_sec_30388.jpg - Authentic: 0.999701976776123
Au_pla_30107.jpg - Authentic: 0.9997765421867371
Au_nat_30181.jpg - Authentic: 0.9992623925209045
Au_pla_30649.jpg - Manipulated: 0.08938989788293839
Au_txt_00034.jpg - Authentic: 0.9913849830627441
Tp_S_NRN_S_N_sec20094_sec20094_01684.jpeg - Authentic: 0.9080571532249451
Au_txt_00020.jpg - Manipulated: 0.002585567766800523
Au_pla_00013.jpg - Manipulated: 0.00027075959951616824
Au_nat_30156.jpg - Authentic: 0.9999922513961792
Au_nat_00095.jpg - Authentic: 0.996400773525238
Au_nat_30630.jpg - Authentic: 0.9928361177444458
Au_sec_30439.jpg - Manipulated: 0.4200306236743927
Au_nat_30624.jpg - Authentic: 0.9999968409538269
Au_arc_30639.jpg - Authentic: 0.9999991059303

Au_pla_30674.jpg - Authentic: 0.9505162239074707
Tp_D_NND_M_O_cha00097_txt00033_10171.jpeg - Authentic: 0.9994560480117798
Au_sec_30564.jpg - Authentic: 0.9999954104423523
Au_sec_30202.jpg - Authentic: 0.9999986290931702
Au_sec_20011.jpg - Manipulated: 0.001004537334665656
Au_pla_30099.jpg - Manipulated: 0.00011921885015908629
Au_sec_30216.jpg - Authentic: 0.9999936819076538
Au_sec_20005.jpg - Manipulated: 0.0030841510742902756
Au_sec_30570.jpg - Authentic: 1.0
Tp_D_CRN_S_N_cha00071_art00092_11783.jpg - Manipulated: 4.1160710679832846e-05
Au_sec_30558.jpg - Authentic: 1.0
Tp_S_NRN_S_N_art00023_art00023_20091.jpg - Manipulated: 0.001672791549935937
Au_nat_30037.jpg - Manipulated: 0.408047080039978
Au_nat_30023.jpg - Authentic: 0.9999069571495056
Au_sec_20039.jpg - Manipulated: 0.0013984816614538431
Au_cha_30550.jpg - Authentic: 0.9999999403953552
Tp_S_NRN_S_N_ind00044_ind00044_10895.jpg - Manipulated: 0.00016184845298994333
Au_pla_30728.jpg - Authentic: 0.9950120449066162
Au_sec_10111.j

Tp_S_NRD_S_N_art10109_art10109_11573.jpg - Manipulated: 0.38191357254981995
Au_sec_30414.jpg - Authentic: 0.9993860125541687
Au_nat_30635.jpg - Authentic: 1.0
Au_arc_30628.jpg - Authentic: 0.9993088841438293
Tp_D_CRN_S_O_ani10180_ani10206_12425.jpg - Manipulated: 0.44696763157844543
Au_pla_00016.jpg - Manipulated: 0.0001537917269160971
Tp_S_NRN_S_N_ani10185_ani10185_12428.jpg - Authentic: 0.8330613970756531
Au_nat_00090.jpg - Manipulated: 0.24324272572994232
Au_nat_30153.jpg - Authentic: 1.0
Au_nat_00084.jpg - Authentic: 0.9644016027450562
Au_nat_30147.jpg - Authentic: 0.9980872869491577
Au_pla_00002.jpg - Authentic: 0.607595682144165
Tp_S_NRN_S_N_txt00083_txt00083_01289.jpeg - Manipulated: 0.06510300189256668
Au_sec_30428.jpg - Authentic: 0.9750807285308838
Tp_S_NRD_S_N_cha10154_cha10154_12266.jpg - Manipulated: 0.45914426445961
Au_nat_30621.jpg - Authentic: 0.9994147419929504
Tp_S_NRN_S_N_pla20088_pla20088_01988.jpeg - Manipulated: 0.0004395093419589102
Au_pla_30328.jpg - Authentic: 

Au_cha_30031.jpg - Authentic: 1.0
Au_arc_30577.jpg - Authentic: 1.0
Tp_S_NRN_S_N_art20050_art20050_01856.jpeg - Manipulated: 0.26915526390075684
Au_cha_30019.jpg - Authentic: 0.9999911189079285
Au_sec_30005.jpg - Authentic: 1.0
Au_pla_20099.jpg - Manipulated: 1.031535475704004e-06
Tp_D_NND_M_N_pla00085_pla00086_00012.jpeg - Manipulated: 0.018164807930588722
Au_nat_30218.jpg - Authentic: 1.0
Tp_S_NRN_S_N_nat00034_nat00034_00954.jpeg - Authentic: 0.999985933303833
Au_sec_30011.jpg - Authentic: 0.9954850077629089
Au_arc_30563.jpg - Authentic: 0.9999999403953552
Au_nat_30595.jpg - Manipulated: 0.05875631049275398
Au_nat_10155.jpeg - Manipulated: 0.05420149490237236
Au_arc_30588.jpg - Authentic: 0.9999998807907104
Tp_S_NRN_S_N_arc20001_arc20001_01454.jpeg - Authentic: 0.5635643005371094
Au_pla_30513.jpg - Authentic: 0.9997795820236206
Au_pla_30275.jpg - Authentic: 0.9773344397544861
Au_pla_20066.jpg - Manipulated: 0.00025722241844050586
Tp_S_NRN_S_N_cha10126_cha10126_12152.jpg - Authentic: 

Au_sec_30601.jpg - Authentic: 1.0
Au_sec_30167.jpg - Authentic: 0.9999732375144958
Tp_S_NRN_S_N_pla00073_pla00073_11226.jpg - Manipulated: 9.273454111280444e-07
Au_nat_30346.jpg - Authentic: 0.9996746778488159
Au_nat_30420.jpg - Authentic: 0.9778192043304443
Tp_D_NNN_M_B_nat10106_nat00097_11352.jpg - Manipulated: 5.098540896142367e-06
Au_cha_00090.jpg - Authentic: 0.9996064901351929
Au_sec_30629.jpg - Authentic: 1.0
Au_cha_00084.jpg - Manipulated: 0.0010025028605014086
Au_nat_30434.jpg - Authentic: 0.9999899864196777
Au_nat_30352.jpg - Manipulated: 0.29401642084121704
Tp_D_CRN_S_N_txt00039_txt00041_10828.jpg - Manipulated: 1.6763307257861015e-06
Au_sec_00098.jpg - Authentic: 0.8291187286376953
Au_nat_30435.jpg - Authentic: 0.9994449615478516
Au_cha_00085.jpg - Authentic: 0.7589267492294312
Au_sec_00099.jpg - Authentic: 0.8178474307060242
Au_nat_30353.jpg - Authentic: 0.9999997019767761
Au_nat_30347.jpg - Authentic: 0.9975727796554565
Tp_S_NRN_S_N_art20082_art20082_02314.jpeg - Manipula